# Plant Disease Detection System
## Multi-Crop Disease Classification using Deep Learning

**Dataset**: Comprehensive plant disease dataset  
**Model**: MobileNetV2 with transfer learning  
**Output**: Trained model (.h5), TFLite model, evaluation metrics

## Setup Instructions
1. Install Python 3.8-3.10
2. Install dependencies: `pip install -r requirements.txt`
3. Ensure dataset is in `final_dataset/` folder with train/val/test subfolders
4. Run cells sequentially


In [ ]:
import sys
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

print("=== PLANT DISEASE DETECTION SETUP ===")
print(f"Python version: {sys.version}")
print(f"Platform: {sys.platform}")

print("\nChecking dependencies...")
dependencies = [
    'tensorflow>=2.8.0,<2.14.0',
    'matplotlib>=3.5.0',
    'scikit-learn>=1.0.0',
    'opencv-python>=4.5.0',
    'seaborn>=0.11.0',
    'numpy>=1.21.0,<2.0.0',
    'pandas>=1.3.0',
    'pillow>=8.0.0'
]

missing_deps = []
for dep in dependencies:
    package_name = dep.split('>=')[0].split('==')[0]
    try:
        __import__(package_name.replace('-', '_'))
        print(f"✓ {package_name} available")
    except ImportError:
        missing_deps.append(dep)
        print(f"✗ {package_name} missing")

if missing_deps:
    print(f"\nInstalling missing dependencies...")
    for dep in missing_deps:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', dep], check=True, capture_output=True)
            print(f"✓ {dep.split('>=')[0]} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ {dep.split('>=')[0]} installation failed")
            print(f"  Error: {e.stderr.decode() if e.stderr else 'Unknown error'}")

print("\n✓ Dependencies check complete!")


In [ ]:
import tensorflow as tf
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

print("=== TENSORFLOW CONFIGURATION ===")
print(f"TensorFlow version: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
print(f"Available GPUs: {len(gpus)}")

use_gpu = False
if gpus:
    print("GPU(s) detected!")
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth enabled")
        
        with tf.device('/GPU:0'):
            test_tensor = tf.constant([1.0, 2.0, 3.0])
            print(f"✓ Test tensor created on GPU: {test_tensor.device}")
        
        try:
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            print("✓ Mixed precision training enabled")
        except Exception as e:
            print(f"Mixed precision setup failed: {e}")
        
        use_gpu = True
        print("✓ GPU training ready!")
        
    except RuntimeError as e:
        print(f"GPU configuration failed: {e}")
        print("Falling back to CPU training")
        use_gpu = False
else:
    print("No GPU detected - using CPU training")

tf.random.set_seed(42)
np.random.seed(42)
print("✓ Setup complete!")


In [ ]:
os.makedirs('models', exist_ok=True)
data_path = 'final_dataset'

print("=== DATASET EXPLORATION ===")
if not os.path.exists(data_path):
    print(f"❌ Dataset folder '{data_path}' not found!")
    print("Please ensure the dataset is in the correct location with train/val/test subfolders")
    raise FileNotFoundError(f"Dataset folder '{data_path}' not found")

total_classes = set()
total_images = 0

for split in ['train', 'val', 'test']:
    split_path = os.path.join(data_path, split)
    if os.path.exists(split_path):
        classes = [d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))]
        split_images = 0
        for class_name in classes:
            class_path = os.path.join(split_path, class_name)
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.JPG', '*.JPEG', '*.png', '*.PNG']:
                image_files.extend(glob.glob(os.path.join(class_path, ext)))
            split_images += len(image_files)
            total_classes.add(class_name)
        print(f"✓ {split.upper()}: {len(classes)} classes, {split_images} images")
        total_images += split_images
    else:
        print(f"⚠️  {split.upper()} folder not found in dataset")

print(f"\n✓ Total dataset: {len(total_classes)} unique classes, {total_images} images")
print("✓ Dataset ready!")


In [ ]:
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(data_path, 'train'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(data_path, 'val'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

steps_per_epoch = train_gen.samples // batch_size
validation_steps = val_gen.samples // batch_size

print("Data generators created")
print(f"Classes: {train_gen.num_classes}")
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")


In [ ]:
if use_gpu:
    with tf.device('/GPU:0'):
        base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
        base.trainable = False
        
        x = GlobalAveragePooling2D()(base.output)
        x = Dropout(0.3)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        
        if tf.keras.mixed_precision.global_policy().name == 'mixed_float16':
            x = tf.keras.layers.Activation('linear', dtype='float32')(x)
        
        out = Dense(train_gen.num_classes, activation='softmax', dtype='float32')(x)
        model = Model(base.input, out)
        
        optimizer = tf.keras.mixed_precision.LossScaleOptimizer(Adam(learning_rate=1e-4))
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
else:
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    base.trainable = False
    
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    out = Dense(train_gen.num_classes, activation='softmax')(x)
    model = Model(base.input, out)
    
    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print(f"Model built: {model.count_params():,} parameters")
model.summary()


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
    ModelCheckpoint('models/best_model.h5', monitor='val_accuracy', save_best_only=True)
]

if use_gpu:
    print("Starting GPU training...")
    with tf.device('/GPU:0'):
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=20,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1
        )
else:
    print("Starting CPU training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=20,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1
    )

model.save('models/final_model.h5')
print("Training completed!")


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Final accuracy: {max(history.history['val_accuracy']):.4f}")


In [ ]:
val_gen.reset()
preds = model.predict(val_gen, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_gen.classes

print("=== CLASSIFICATION REPORT ===")
class_names = list(val_gen.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=class_names))

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

accuracy = np.mean(y_pred == y_true)
print(f"Overall Accuracy: {accuracy:.4f}")


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('models/model.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved: {len(tflite_model) / 1024:.2f} KB")
print("Ready for mobile deployment!")


In [ ]:
print("=== CREATING IoT MODEL ===")

try:
    with open('class_indices.json', 'r') as f:
        all_classes = json.load(f)
    print(f"✓ Loaded {len(all_classes)} classes from class_indices.json")
except FileNotFoundError:
    print("⚠️  class_indices.json not found, using default classes")
    all_classes = {}

important_classes = [
    "paddy_normal",
    "paddy_blast", 
    "paddy_brown_spot",
    "wheat_Healthy",
    "wheat_septoria",
    "Tomato_healthy"
]

print(f"Reduced classes from {len(all_classes)} to {len(important_classes)}")

def create_ultra_small_model():
    print("Creating ultra-small CNN model...")
    
    img_size = 64
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(img_size, img_size, 3)),
        
        tf.keras.layers.Conv2D(8, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(important_classes), activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model, img_size

iot_model, img_size = create_ultra_small_model()

print(f"Model parameters: {iot_model.count_params():,}")
print(f"Input size: {img_size}x{img_size}")
print(f"Classes: {len(important_classes)}")

iot_model.summary()
